In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
#----------------Read_data----------------
train_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv") 

In [ ]:
train_text_len = [len(row.split())for row in train_data['text']]
test_text_len = [len(row.split())for row in test_data['text']]

In [ ]:
train_text_len = np.array(train_text_len)/max(train_text_len)
test_text_len = np.array(test_text_len)/max(test_text_len)

In [ ]:
print("Train_data_dim:{}".format(train_data.shape))
print("Test data_dim:{}".format(test_data.shape))

In [ ]:
#--------Sample_train_data---------------
train_data.head()

In [ ]:
#-----------Sample_test_data------------------
test_data.head()

In [ ]:
#-------------Check_class_imbalance-------------------
sns.countplot(x ='target',data = train_data)

The above graph shows that, the distribution of comments across each class is balanced.
Imbalance handling process is not required.

In [ ]:
target = train_data["target"]
train_data = train_data['text']

In [ ]:
train_data

In [ ]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string

In [ ]:
def text_preprocessor(text):
    #-------------Remove_stop_words----------------
    stop_free_text = " ".join([word.lower() for word in word_tokenize(text) if word not in stop_words])
    #------------Remove_numeric_values--------------
    digit_free_text = res = "".join(filter(lambda x: not x.isdigit(), stop_free_text))
    #-------------Remove_punctuations---------------
    punct_free_text = " ".join([word for word in word_tokenize(digit_free_text) if word not in list(string.punctuation)])
    #-------------Lemmatize_text--------------------
    lemmatised_text = " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(punct_free_text)])
    return lemmatised_text
                               

In [ ]:
train_df = pd.DataFrame([])
processed_text = []
for text_data in train_data:
    processed_text.append(text_preprocessor(text_data))
    
train_df['text'] = processed_text
train_df['text_len'] = train_text_len

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_df,target,test_size=0.20,random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features=500,min_df=1)
train_X = vectorizer.fit_transform(X_train['text'].tolist())
test_X = vectorizer.transform(X_test['text'].tolist())

In [ ]:
train_X = pd.DataFrame(train_X.toarray())
test_X = pd.DataFrame(test_X.toarray())
train_X['text_len'] = list(X_train['text_len'])
test_X['text_len'] = list(X_test['text_len'])

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
#--------------Train_ML_model---------------------
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
model = MultinomialNB()
model = RandomForestClassifier(random_state = 42)
model.fit(train_X,y_train)
model

In [ ]:
#-----------Generate_predictions---------------
predictions = model.predict(test_X)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print("Accuracy:{}".format(accuracy_score(y_test,predictions)))
print(classification_report(y_test,predictions))

In [ ]:
#--------Predict_classes_for_test_dataset--------------
test_text = test_data['text']

In [ ]:
test_df = pd.DataFrame([])
processed_text = []
for text_data in test_text:
    processed_text.append(text_preprocessor(text_data))
    
test_df['text'] = processed_text

In [ ]:
test_data = vectorizer.transform(test_df['text'].tolist())
test_data = pd.DataFrame(test_data.toarray())
test_data['test_len'] = test_text_len

In [ ]:
#-------------generate_test_predictions-----------------
test_preds = model.predict(test_data)

In [ ]:
temp = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
temp['target'] = list(test_preds)

In [ ]:
temp.columns

In [ ]:
temp.to_csv('Submission_file_v7.csv',index=False)